In [46]:
import pandas as pd
import requests
import json
import time

In [24]:
events = []

eventsFilePath = "events/events.json"

with open(eventsFilePath, "r") as f:
    events_data = json.load(f)

events = events_data["pageProps"]["events"]

print("Event IDs and Names:")
print("====================")

vctEvents = [event for event in events if event["shortName"].startswith("VCT")]

vctEventIds = [event["id"] for event in vctEvents]

print("VCT Event IDs:")
print(vctEventIds)
print(f"\nTotal VCT events found: {len(vctEventIds)}")

def format_to_csv(df, cols, path):
    df = pd.DataFrame(df)
    df.columns = cols
    df.index = pd.RangeIndex(len(df.index))
    df.to_csv(path)


Event IDs and Names:
VCT Event IDs:
[5574, 5461, 5469, 5465, 5474, 5351, 5232, 5228, 5230, 5233]

Total VCT events found: 10


In [ ]:
all_match_events = []
all_event_locations = []
all_player_economies = []

def fetchEventData(eventId):
    url = f"https://be-prod.rib.gg/v1/series?completed=true&take=50&eventIds%5B%5D={eventId}"
    JSONContent = requests.get(url).json()
    matchIDs = []
    matchMaps = []
    
    matchEvents = []
    eventLocations = []
    playerEconomies = []
    
    for data in JSONContent["data"]:
        for match in data["matches"]:
            matchIDs.append(match["id"])
            matchMaps.append(match["map"])
    
    for matchID, matchMap in zip(matchIDs, matchMaps):
        content = requests.get("https://be-prod.rib.gg/v1/matches/" + str(matchID) + "/details").json()
        for event in content["events"]:
            matchEvents.append([matchMap["name"],
                                content["id"],
                                event["roundId"],
                                event["roundNumber"],
                                event["roundTimeMillis"],
                                event["eventType"],
                                event["killId"],
                                event["tradedByKillId"],
                                event["tradedForKillId"],
                                event["bombId"],
                                event["resId"],
                                event["playerId"],
                                event["assists"],
                                event["referencePlayerId"],
                                event["damageType"],
                                event["weaponId"],
                                event["ability"],
                                eventId])

        for location in content["locations"]:
            eventLocations.append([matchMap["name"],
                                location["roundNumber"],
                                location["playerId"],
                                location["roundTimeMillis"],
                                location["locationX"],
                                location["locationY"],
                                location["viewRadians"],
                                eventId])

        for economy in content["economies"]:
            playerEconomies.append([matchMap["name"],
                                    economy["roundId"],
                                    economy["roundNumber"],
                                    economy["playerId"],
                                    economy["agentId"],
                                    economy["score"],
                                    economy["weaponId"],
                                    economy["armorId"],
                                    economy["remainingCreds"],
                                    economy["spentCreds"],
                                    economy["loadoutValue"],
                                    economy["survived"],
                                    economy["kast"],
                                    eventId])
    
    return matchEvents, eventLocations, playerEconomies

In [23]:
for eventId in vctEventIds:
    print(f"Fetching data for event ID: {eventId}")
    match_events, event_locations, player_economies = fetchEventData(eventId)
    all_match_events.extend(match_events)
    all_event_locations.extend(event_locations)
    all_player_economies.extend(player_economies)
    print(f"Data for event ID {eventId} fetched.")

matchEvents_dataset_columns = ["Map", "Match_Id", "Round_ID", "Round_Number", "Round_Time", 
                            "Event_Type", "Kill_ID", "Traded_by_ID", "Traded_for_ID", 
                            "Bomb_Status", "Resurrection_ID", "Player_ID", "Assists", 
                            "Reference_Player_ID", "Damage_Type", "Weapon_ID", "Ability", "Event_ID"]

eventLocations_dataset_columns = ["Map", "Round_Number", "Player_ID", "Round_Time", 
                               "X", "Y", "View_Radians", "Event_ID"]

playerEconomies_dataset_columns = ["Map", "Round_ID", "Round_Number", "Player_ID", "Agent_ID", 
                                "Round_Score", "Weapon_ID", "Armor_ID", "Remaining_Credits", 
                                "Spent_Credits", "Loadout_Value", "Survived", "kast", "Event_ID"]

format_to_csv(all_match_events, matchEvents_dataset_columns, "datasets/valorant-match-events.csv")
format_to_csv(all_event_locations, eventLocations_dataset_columns, "datasets/valorant-event-locations.csv")
format_to_csv(all_player_economies, playerEconomies_dataset_columns, "datasets/valorant-player-economies.csv")

print("All data fetched and saved to consolidated files successfully.")

Fetching data for event ID: 5574
Data for event ID 5574 fetched.
Fetching data for event ID: 5461
Data for event ID 5461 fetched.
Fetching data for event ID: 5469
Data for event ID 5469 fetched.
Fetching data for event ID: 5465
Data for event ID 5465 fetched.
Fetching data for event ID: 5474
Data for event ID 5474 fetched.
Fetching data for event ID: 5351
Data for event ID 5351 fetched.
Fetching data for event ID: 5232
Data for event ID 5232 fetched.
Fetching data for event ID: 5228
Data for event ID 5228 fetched.
Fetching data for event ID: 5230
Data for event ID 5230 fetched.
Fetching data for event ID: 5233
Data for event ID 5233 fetched.
All data fetched and saved to consolidated files successfully.


In [48]:
df = pd.read_csv("datasets/valorant-match-events.csv")

matches_by_map = df.groupby("Map")["Match_Id"].unique()

kill_data = {}

for map_name, match_ids in matches_by_map.items():
    print(f"Map: {map_name}")
    map_data = []
    
    for match_id in match_ids:
        try:
            url = f"https://be-prod.rib.gg/v1/x/players/kill-death-locations-by-match?matchId={match_id}"
            print(f"  Fetching data for match ID: {match_id}")
            
            response = requests.get(url)
            response.raise_for_status()
            match_data = response.json()
            
            map_data.append(match_data)
            
            # Be nice to the API
            time.sleep(0.5)
            
        except Exception as e:
            print(f"  Error fetching data for match {match_id}: {e}")

    kill_data[map_name] = map_data

    with open(f"datasets/map_kill_data/{map_name.lower()}_kill_data.json", "w") as f:
        json.dump(kill_data, f)
        
with open("datasets/all_maps_kill_data.json", "w") as f:
    json.dump(kill_data, f)



Map: Abyss
  Fetching data for match ID: 193474
  Fetching data for match ID: 192486
  Fetching data for match ID: 192477
  Fetching data for match ID: 191902
  Fetching data for match ID: 191073
  Fetching data for match ID: 190509
  Fetching data for match ID: 189703
  Fetching data for match ID: 189109
  Fetching data for match ID: 189115
  Fetching data for match ID: 189244
  Fetching data for match ID: 187493
  Fetching data for match ID: 187490
  Fetching data for match ID: 191104
  Fetching data for match ID: 191033
  Fetching data for match ID: 190659
  Fetching data for match ID: 190539
  Fetching data for match ID: 190650
  Fetching data for match ID: 189222
  Fetching data for match ID: 188338
  Fetching data for match ID: 188175
  Fetching data for match ID: 188171
  Fetching data for match ID: 187458
  Fetching data for match ID: 191084
  Fetching data for match ID: 190512
  Fetching data for match ID: 190359
  Fetching data for match ID: 190361
  Fetching data for match I